# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [306]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [307]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202102.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202102.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [308]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[가리비국산]가리비국산,5kg,보통(3등),23000,경남 남해,일반,20210201
1,[가리비국산]가리비국산,5kg,보통(3등),24000,경남 남해,일반,20210201
2,[가리비국산]가리비국산,5kg,보통(3등),25000,"경남 마산(고성,진해)",일반,20210201
3,[가리비국산]가리비국산,5kg,보통(3등),24000,"경남 마산(고성,진해)",일반,20210201
4,[가오리수입]가오리수입,3kg,보통(3등),22300,수입 베트남,일반,20210201


In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388781 entries, 0 to 388780
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     388781 non-null  object
 1   단위      388781 non-null  object
 2   등급      388781 non-null  object
 3   가격      388781 non-null  int64 
 4   산지      387223 non-null  object
 5   친환경구분   388781 non-null  object
 6   입력일     388781 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 20.8+ MB


In [310]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1558
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [311]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [312]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [313]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '4등', '5등', '6등', '7등',
       '8등'],
      dtype='object')

In [314]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [315]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [316]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,일반,20210201,3
1,[가리비국산]가리비국산,5kg,24000,경남 남해,일반,20210201,3
2,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",일반,20210201,3
3,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",일반,20210201,3
4,[가오리수입]가오리수입,3kg,22300,수입 베트남,일반,20210201,3
...,...,...,...,...,...,...,...
387218,[토마토]대저토마토,2.5kg,8000,부산 강서구,일반,20210227,7
387219,[토마토]토마토(기타),2.5kg,4500,부산 강서구,일반,20210227,7
387220,[토마토]토마토(기타),2.5kg,14000,부산 강서구,일반,20210227,7
387221,[토마토]토마토(기타),2.5kg,6000,부산 강서구,일반,20210227,7


### eco

In [317]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [318]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,20210201,3
1,[가리비국산]가리비국산,5kg,24000,경남 남해,normal,20210201,3
2,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,20210201,3
3,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",normal,20210201,3
4,[가오리수입]가오리수입,3kg,22300,수입 베트남,normal,20210201,3
...,...,...,...,...,...,...,...
387218,[토마토]대저토마토,2.5kg,8000,부산 강서구,normal,20210227,7
387219,[토마토]토마토(기타),2.5kg,4500,부산 강서구,normal,20210227,7
387220,[토마토]토마토(기타),2.5kg,14000,부산 강서구,normal,20210227,7
387221,[토마토]토마토(기타),2.5kg,6000,부산 강서구,normal,20210227,7


### reg_date

In [319]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [320]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [321]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,2021-02-01,3
1,[가리비국산]가리비국산,5kg,24000,경남 남해,normal,2021-02-01,3
2,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,2021-02-01,3
3,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",normal,2021-02-01,3
4,[가오리수입]가오리수입,3kg,22300,수입 베트남,normal,2021-02-01,3
...,...,...,...,...,...,...,...
387218,[토마토]대저토마토,2.5kg,8000,부산 강서구,normal,2021-02-27,7
387219,[토마토]토마토(기타),2.5kg,4500,부산 강서구,normal,2021-02-27,7
387220,[토마토]토마토(기타),2.5kg,14000,부산 강서구,normal,2021-02-27,7
387221,[토마토]토마토(기타),2.5kg,6000,부산 강서구,normal,2021-02-27,7


### price

In [322]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [323]:
df2['scale'].unique()

array([    5.  ,     3.  ,    40.  ,     4.5 ,     8.  ,    10.  ,
          11.  ,    20.  ,     6.  ,     2.  ,     4.  ,     3.1 ,
           5.4 ,     0.15,     0.05,     0.2 ,     2.3 ,     1.  ,
           0.8 ,     0.9 ,     7.  ,     3.5 ,     1.8 ,     1.9 ,
           1.7 ,     0.6 ,     5.1 ,     0.5 ,     3.6 ,     3.2 ,
          21.5 ,    23.  ,     7.5 ,     9.  ,     1.5 ,     1.3 ,
           1.4 ,     1.6 ,    15.  ,     2.4 ,    12.  ,     2.2 ,
          18.  ,     2.5 ,    13.  ,     0.06,     0.7 ,     9.5 ,
         200.  ,     3.7 ,     3.8 ,     3.4 ,     3.3 ,     2.7 ,
           5.5 ,    14.  ,     9.6 ,     6.1 ,     5.2 ,    22.5 ,
          12.2 ,    36.  ,     2.1 ,    14.1 ,     2.74,    12.5 ,
           4.8 ,    17.  ,   103.  ,     1.2 ,    34.  ,    55.  ,
           4.4 ,     4.7 ,     9.4 ,    18.8 ,     1.1 ,     4.3 ,
           8.5 ,     5.01,     9.8 ,    11.5 ,     1.52,     4.2 ,
           8.01,    16.  ,    30.  ,     4.6 ,     9.2 ,    21

In [324]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     387223.00000
mean       27080.45396
std        99011.55019
min            0.00000
25%         7900.00000
50%        16000.00000
75%        34000.00000
max     30012000.00000
Name: price, dtype: float64

In [325]:
df2 = df2.drop(df2[df2['price']<500].index)

In [326]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5.00000,23000,경남 남해,normal,2021-02-01,3
1,[가리비국산]가리비국산,5.00000,24000,경남 남해,normal,2021-02-01,3
2,[가리비국산]가리비국산,5.00000,25000,"경남 마산(고성,진해)",normal,2021-02-01,3
3,[가리비국산]가리비국산,5.00000,24000,"경남 마산(고성,진해)",normal,2021-02-01,3
4,[가오리수입]가오리수입,3.00000,22300,수입 베트남,normal,2021-02-01,3
...,...,...,...,...,...,...,...
387218,[토마토]대저토마토,2.50000,8000,부산 강서구,normal,2021-02-27,7
387219,[토마토]토마토(기타),2.50000,4500,부산 강서구,normal,2021-02-27,7
387220,[토마토]토마토(기타),2.50000,14000,부산 강서구,normal,2021-02-27,7
387221,[토마토]토마토(기타),2.50000,6000,부산 강서구,normal,2021-02-27,7


In [327]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [328]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[딸기]설향             13074
[오이]백다다기            8571
[딸기]설향(딸기)          7155
[시금치]시금치            6087
[딸기]딸기(설향)          5160
                   ...  
[기타과일]패션푸르츠(수입)        1
[감자]감자(추백)             1
[양채류]향나물               1
[무우]마대무우               1
[양배추]양배추(기타)           1
Name: prd, Length: 1224, dtype: int64

In [329]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [330]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [331]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [332]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가지 각굴 갈치 감 감귤 감말랭이 감자 갑오징어 갓 강낭콩 개조개 
건고추 건채류 게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고사리 고추 고추잎 곤달비 
곤드레나물 골뱅이 곶감 과실류 과일 과일류 굴 귤 그레이프푸룻 근대 근채류 금감 김 깻잎 꼴뚜 
꽁치 꽈리고추 낙지 냉이 논우렁이 눈볼대 느 다래 단감 달래 당귀 당근 대구 대구포 대추 
대파 더덕 도라지 돗나물 동자개 동죽 두류 두릅 두리안 딸 땅콩 람부탄 레드쉬 레몬 리치 
마 마늘 만가닥 만감 망고 매생이 머위대 메 메론 명태 명태곤니 명태알 명태포 모시조개 모자반 
목이 무 무순 무우 무청 문어바지락 미꾸라지 미나리 미더덕 미역 민들레 민물돔 민물새우 민어 바나나 
바지락 박 밤 방어 방울토마토 방풍 방풍나물 배 배추 백새우 백조 백합 뱀장어 버섯 버섯류 
버찌 벗 병어 보리 봄동 봄동배추 봉지굴 부세 부지갱이 부추 붉은고추 붕어 브로커리 브로코리 블루베리 
비 비트 빈스 사과 삼 삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 샐러리 생강 
생고추 석류 선동오징어 세러리 세발나물 세트 셀러리 소라 속새 숙주나물 숫꽃게 시금치 쌈채류 쑥 쑥갓 
씀바귀 아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 알 알로애 알배 암꽃게 야자 양배추 양상추 양송이 
양채 양채류 양파 어류 얼갈이 얼갈이배추 연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오렌지 오만둥이 
오이 오징어 오징어다리 오징어몸통 오징어원양 오징어채 완두콩 용과 우럭조개 우렁 우렁쉥이 우렁이 우뭇가시리 우엉 유채 
임연 잉어 자라 자몽 재첩 적겨자 적근대 적새우 적채 전복 절단 절단낙지 조개살 조미제품 죽순 
중하 쪽파 쭈꾸미 참꼬막 참나물 참다래 참당귀 참돔 참외 참조 채소 청경채 청어 체리 총각무 
춘채 취나물 치커리 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콜라비 콩 콩나물 키위 키조개 키조개날개 
토란 토마토 톳 파 파래 파래김 파세리 파인애플 파프리카 팽이 팽이버섯 포도

In [333]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [334]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [335]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [338]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#붉은고추 -> 홍고추로 통합 
df2.loc[df2["prd"]=="붉은고추","prd"] = "홍고추"
#비 - > 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯 "
#선동오징어 -> 오징어로 통합
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"
# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피마자잎","prd"] = "피마자"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)

In [340]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array(['방어', '참돔', '토란', '골뱅이', '꽃게', '우렁이', '보리', '세발나물', '민들레', '알로에',
       '호두', '춘채', '레디쉬', '목이', '파래', '건채류', '피마자', '곤드레나물', '두류', '강낭콩',
       '고구마순'], dtype=object)

In [341]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_6216/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [342]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [343]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [344]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,234.00000
mean,1652.10684
std,4690.45422
min,1.00000
25%,9.00000
50%,87.00000
75%,779.25000
max,42726.00000


In [345]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [346]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

딸기      42726
버섯      33838
오이      17965
토마토     17865
감귤      15734
        ...  
부세         54
미꾸라지       52
아귀         52
새조개        50
콩나물        50
Name: prd, Length: 131, dtype: int64

In [347]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [348]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([    5.  ,     4.  ,     2.  ,    10.  ,    40.  ,     4.5 ,
           3.6 ,    16.  ,    18.  ,     8.  ,     5.01,    57.  ,
           9.  ,    28.  ,    26.  ,     7.  ,    25.  ,    46.  ,
          50.  ,     3.  ,    13.  ,     2.5 ,    17.  ,    15.  ,
           6.5 ,     7.5 ,    12.  ,    76.  ,    11.  ,     3.1 ,
           1.  ,    20.  ,     1.5 ,     0.5 ,     0.8 ,     1.2 ,
           6.  ,     9.6 ,     7.3 ,    23.  ,     2.4 ,    24.  ,
           5.4 ,     5.1 ,     5.5 ,     2.2 ,    14.  ,     4.7 ,
          12.5 ,     4.2 ,     3.5 ,     8.5 ,     4.6 ,     7.8 ,
          11.5 ,     0.1 ,     6.75,     0.2 ,     0.3 ,    12.8 ,
           9.5 ,     3.2 ,     6.7 ,     4.4 ,     2.7 ,     3.8 ,
           0.25,     0.13,     0.05,    10.5 ,     0.68,     0.7 ,
           0.9 ,     0.78,     0.15,     0.06,    21.  ,    22.  ,
          19.  ,     1.8 ,     1.6 ,    12.3 ,     1.3 ,     4.3 ,
           0.16,     0.12,     2.3 ,     0.6 ,     3.3 ,    55

In [349]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1155,가지,57.00000,353400,경남 진주시,certified,2021-02-01,2,6200.00000
2845,가지,57.00000,353400,경남 진주시,certified,2021-02-03,1,6200.00000
3414,가지,50.00000,310000,경남 진주시,certified,2021-02-10,1,6200.00000
4664,가지,76.00000,471200,경남 진주시,certified,2021-02-15,9,6200.00000
80519,바지락,55.00000,12000,수입 중국,normal,2021-02-06,3,218.18000
84486,당근,200.00000,20000,제주 제주시,normal,2021-02-02,3,100.00000
84600,당근,200.00000,15000,제주 제주시,normal,2021-02-10,3,75.00000
84765,당근,200.00000,25000,제주 제주시,normal,2021-02-02,2,125.00000
84903,당근,200.00000,18000,제주 제주시,normal,2021-02-10,2,90.00000
85171,당근,200.00000,28000,제주 제주시,normal,2021-02-02,1,140.00000


In [350]:
# 스케일 오표기된 항목 수정
# 가지는 가격이 평균보다 비싸긴 하나 거래 불가능한 정도는 아니라 판단
# 당근 가격은 같은 지역에서 생산된 평균가와 10배 차이 나므로 오표기라 판단, 수정
df3.loc[(df3['prd']=='당근') & (df3['scale']==200),'scale'] = 20


In [351]:
# 나머지는 이상치로 판단하고 모두 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='가지')].index)

In [352]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1155,가지,57.00000,353400,경남 진주시,certified,2021-02-01,2,6200.00000
2845,가지,57.00000,353400,경남 진주시,certified,2021-02-03,1,6200.00000
3414,가지,50.00000,310000,경남 진주시,certified,2021-02-10,1,6200.00000
4664,가지,76.00000,471200,경남 진주시,certified,2021-02-15,9,6200.00000


### origin

In [353]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경남 남해', '경남 마산(고성,진해)', '경남 통영', '전남 여수', '경남 삼천포', '수입 중국',
       '수입 일본', '수입 러시아', '수입 미국', '전북 군산', '경남 밀양시', '경남 진주시', '경남 고성군',
       '경북 상주시', '대구 달성군', '경기 여주시', '전북 군산시', '전북 전주시', '경남 창녕군',
       '경남 함안군', '경북 고령군', '경남 김해시', '광주시 광산구', '전북 임실군', '광주 광산구',
       '경기 여주군', '경기 안성시 일죽면', '경남 거제시', '경북 성주군', '경남 합천군', '전북 익산시',
       '전남 나주시', '경남 창원시 의창구', '경남 창원시', '경기 양평군', '충남 논산시', '경남',
       '경남 진주시 금곡면', '전남 광양시', '서울', '경남 의령군', '전북 완주군', '충남 부여군',
       '경남 밀양시 단장면', '대전 대덕구', '전북 군산시 개정면', '광주', '충남 아산시', '충남 천안시',
       '충남 홍성군', '서울 송파구', '충남 서천군', '강원 춘천시', '경기 광주시', '경기 하남시',
       '전남 고흥(나로도)', '충남 서산', '국내산', '수입 세네갈', '제주 제주도', '수입 기타',
       '제주 제주시', '제주 서귀포시', '제주', '제주 북제주군', '제주 서귀포시(2)', '제주특별자치도 서귀',
       '제주 남제주군', '제주도 서귀포시', '제주도 제주시', '경남 산청군', '전남 고흥군', '경기 안성시',
       '전남 무안군', '전남 담양군', '제주 서귀포시 대정읍', '제주 제주시 조천읍', '제주 제주시 애월읍',
       '경북 청송군', '경기 평택시', '전남 장성군', '전남 해남군', '서울특별시 강남구', '제주 서귀포',
       '충남 서산시', '경북 구미시', '전

In [354]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,가리비,5.00000,23000,경남 남해,normal,2021-02-01,3,4600.00000,경남,남해,NaN,NaN
1,가리비,5.00000,24000,경남 남해,normal,2021-02-01,3,4800.00000,경남,남해,NaN,NaN
2,가리비,5.00000,25000,"경남 마산(고성,진해)",normal,2021-02-01,3,5000.00000,경남,"마산(고성,진해)",NaN,NaN
3,가리비,5.00000,24000,"경남 마산(고성,진해)",normal,2021-02-01,3,4800.00000,경남,"마산(고성,진해)",NaN,NaN
4,가리비,5.00000,24000,"경남 마산(고성,진해)",normal,2021-02-02,3,4800.00000,경남,"마산(고성,진해)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
385361,표고버섯,14.00000,91000,경기 의왕시,normal,2021-02-27,9,6500.00000,경기,의왕시,NaN,NaN
385362,표고버섯,15.00000,93750,경기 의왕시,normal,2021-02-27,9,6250.00000,경기,의왕시,NaN,NaN
385363,표고버섯,15.00000,97500,경기 의왕시,normal,2021-02-27,9,6500.00000,경기,의왕시,NaN,NaN
385364,표고버섯,14.00000,98000,경기 의왕시,normal,2021-02-27,9,7000.00000,경기,의왕시,NaN,NaN


In [355]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [356]:
df4['state'].unique()

array(['경남', '전남', '수입', '전북', '경북', '대구', '경기', '광주시', '광주', '충남', '서울',
       '대전', '강원', '국내산', '제주', '제주특별자치도', '제주도', '서울특별시', '부산시', '강원도',
       '충북', '전주시', '서울시', '세종', '미국', '경기도', '인천시', '인천', '전라북도', '부산',
       '성남시', '중국', '대만', '수입산', '울산', '포항시', '청주시', '충청남도', '경상북도',
       '세종시', '가락동', '경북/문경시', '일본', '원양산', '경상남도', '인천광역시', '뉴질랜드', '페루',
       '칠레', '필리핀', '인도네시아', '베트남', '이란', '이탈리아', '과테말라', '코스타리카', '콜롬비아',
       '에쿠아도르', '에콰돌', '에콰도르', '멕시코', '충북청주시상당구', '스페인', '태국', '타이',
       '러시아', '이스라엘'], dtype=object)

In [357]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [358]:
df4["state"].unique()

array(['경남', '전남', '수입', '전북', '경북', '대구', '경기', '광주', '충남', '서울', '대전',
       '강원', '국내산', '제주', '제주특별자치', '서울특별', '부산', '충북', '전주', '세종', '미국',
       '인천', '전라북', '성남', '중국', '대만', '수입산', '울산', '포항', '청주', '충청남',
       '경상북', '가락동', '경북/문경', '일본', '원양산', '경상남', '인천광역', '뉴질랜드', '페루',
       '칠레', '필리핀', '인', '베트남', '이란', '이탈리아', '과테말라', '코스타리카', '콜롬비아',
       '에쿠아', '에콰돌', '에콰', '멕', '충북청주', '스페인', '태국', '타이', '러', '이스라엘'],
      dtype=object)

In [359]:
df4[df4["state"]=="인"]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city
326135,포도,8.00000,20000,인도네시아,normal,2021-02-24,1,2500.00000,인,NaN
326235,포도,8.00000,20000,인도네시아,normal,2021-02-26,1,2500.00000,인,NaN
326349,포도,8.00000,20000,인도네시아,normal,2021-02-27,1,2500.00000,인,NaN
341489,바나나,13.00000,16000,인도네시아,normal,2021-02-01,1,1230.77000,인,NaN
341634,바나나,13.00000,16000,인도네시아,normal,2021-02-02,1,1230.77000,인,NaN
341733,바나나,13.00000,16000,인도네시아,normal,2021-02-03,1,1230.77000,인,NaN
341860,바나나,13.00000,16000,인도네시아,normal,2021-02-04,1,1230.77000,인,NaN
341961,바나나,13.00000,16000,인도네시아,normal,2021-02-05,1,1230.77000,인,NaN
342089,바나나,13.00000,16000,인도네시아,normal,2021-02-06,1,1230.77000,인,NaN
342182,바나나,13.00000,16000,인도네시아,normal,2021-02-08,1,1230.77000,인,NaN


In [360]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [361]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [362]:
df4['state'].unique()

array(['경남', '전남', '수입', '전북', '경북', '대구', '경기', '광주', '충남', '서울', '대전',
       '강원', '국산', '제주', '부산', '충북', '세종', '인천', '울산', '원양산'],
      dtype=object)

In [363]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [364]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [365]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [366]:
df4['city'].unique()

array(['남해', '마산(고성,진해)', '통영', '여수', '삼천포', '중국', '일본', '러', '미국', '산',
       '밀양', '진주', '고성', '상주', '대구', '여주', '전주', '창녕', '함안', '고령', '김해',
       '광산구', '임실', '안성', '거제', '성주', '합천', '익산', '나주', '창원', '양평', '논산',
       '미상', '광양', '의령', '완주', '부여', '대전', '아산', '천안', '홍성', '송파구', '서천',
       '춘천', '광주', '하남', '고흥(나로도)', '서산', '세네갈', '제주', '기타', '산청', '고흥',
       '무안', '담양', '청송', '평택', '장성', '해남', '서울', '구미', '남원', '보성', '평창',
       '이천', '부산', '강릉', '보령', '당진', '하동', '태안', '김제', '인제', '홍천', '부안',
       '옥천', '예천', '정선', '안동', '금산', '강서구', '영주', '김천', '횡성', '세종', '봉화',
       '진천', '원주', '곡성', '공주', '베트남', '타이', '포천', '구리', '예산', '고양', '중구',
       '파주', '바레인', '파키스탄', '장항(서천)', '강화', '영암', '고창', '음성', '청양', '남구',
       '마산', '서구', '순천', '화순', '강진', '영동', '충주', '양주', '강동구', '광명', '남양주',
       '성남', '용인', '양천구', '제천', '동대문구', '흥', '의왕', '경산', '도봉구', '영광',
       '화성', '의정부', '대만', '중랑구', '영양', '안산', '신안', '괴산', '울산', '진도', '철원',
       '포항', '가평', '함평', '영덕', '청원', '미금', '청

In [367]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
## 광역시

df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [368]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array([], dtype=object)

In [369]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_6216/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [370]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [371]:
cities.to_csv('../cities.csv')

In [372]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [373]:
df5.to_csv('../data_proceed/202102_proceed.csv')